In [1]:
!pip install timm
import os
import torch
import zipfile
import pandas as pd
import numpy as np
import timm as tm
from PIL import Image
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from PIL import Image
from google.colab.patches import cv2_imshow
%matplotlib inline

     |████████████████████████████████| 296kB 10.1MB/s 


In [2]:
labels = {
    0: 'badminton',
    1: 'baseball',
    2: 'basketball',
    3: 'boxing',
    4: 'chess',
    5: 'cricket',
    6: 'fencing',
    7: 'football',
    8: 'formula1',
    9: 'gymnastics',
    10: 'hockey',
    11: 'ice_hockey',
    12: 'kabaddi',
    13: 'motogp',
    14: 'shooting',
    15: 'swimming',
    16: 'table_tennis',
    17: 'tennis',
    18: 'volleyball',
    19: 'weight_lifting',
    20: 'wrestling',
    21: 'wwe'
}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with zipfile.ZipFile('/content/drive/MyDrive/test1.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

In [5]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

device = get_default_device()
device

device(type='cuda')

path_to_model - Путь до модели(модель необходимо загрузить отдельно)

path_to_images - Путь до папки с изображениями

In [ ]:
def infer(path_to_model, path_to_images): 
  model = tm.create_model('efficientnet_b3a', pretrained=False, num_classes=22)
  model = to_device(model, device) 
  model_s = torch.load(path_to_model) 
  model.load_state_dict(model_s, strict=False) 
  model.eval()

  pil2tensor = transforms.ToTensor()
  images = []
  predictions = []
  index = []
  i = 0
  for filename in os.listdir(path_to_images):
    img = Image.open(path_to_images + '/' + filename)
    xb = img 
    xb =  xb.resize((224, 224))
    xb = pil2tensor(xb)
    xb = to_device(xb, device)
    preds = model(xb.reshape(1, 3, 224, 224))
    prediction = preds[0]
    _, preds = torch.max(prediction, dim=0)
    preds.cpu()
    label = labels[preds.item()]
    images.append(filename)
    index.append( int(filename.split('.')[0]) )
    predictions.append(label)
  output = pd.DataFrame(data={"index":index,"image":images,"label":predictions})
  sorted_df = output.sort_values(by=["index"], ascending=True) 
  sorted_df = sorted_df.drop(columns="index")
  sorted_df.to_csv(path_or_buf="submission.csv", index=False, quoting=3)

from google.colab import files
files.upload()

files.download('efficientnet_b3a_86.pt')
infer('/content/efficientnet_b3a_86.pt', '/content/test1')